In [ ]:
import torch

In [ ]:
import zipfile

In [ ]:
with zipfile.ZipFile("celebA_resize3.zip", 'r') as zip_ref:
    zip_ref.extractall("Notebooks/")

In [ ]:
import glob
files = glob.glob("Notebooks/celebA_resize3/*")

In [ ]:
len(files)

In [ ]:
!git clone https://github.com/mar10nJohns1/AlgorithmicFairness

In [1]:
from __future__ import print_function
#%matplotlib inline
import argparse
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import pandas as pd
import data_utils.data_utils_celeba_pytorch5 as data_utils
import data_utils.network_tuning23 as network_tuning
from IPython.display import clear_output
from skimage.io import imread
from skimage.transform import resize
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import pandas as pd
import pickle
# Load functions
#import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
from torch.nn import Linear, GRU, Conv2d, Dropout2d, MaxPool2d, BatchNorm2d
from torch.nn.functional import relu, elu, relu6, sigmoid, tanh, softmax

In [2]:
dataroot = 'Data/'
TRAIN_PATH = dataroot+"train.csv"
VALID_PATH = dataroot + "valid.csv" 
TEST_PATH = dataroot + "test.csv" 

In [3]:
IMAGE_SHAPE = [73,60,3]
# Paths to data
# Root directory for dataset
IMAGE_PATHS = "Notebooks/celebA_resize3/"
TARGET_COL = 'Smiling'
NUM_CLASSES = 2

# train holds both X (input) and t (target/truth)
data_train = data_utils.CelebADataset(TRAIN_PATH,IMAGE_PATHS,IMAGE_SHAPE,TARGET_COL)
data_valid = data_utils.CelebADataset(VALID_PATH,IMAGE_PATHS,IMAGE_SHAPE,TARGET_COL)

#tuning the network round 1
df = pd.DataFrame(index=list(range(1,13)), columns=['layers','activations','conv_out_channels','kernel_size','conv_stride','maxpool'\
                                                    ,'dropout','batchnorm','optimizer','learning_rate','weight_decay','batch_size','num_epochs'\
                                                    ,'net','train_loss','train_accs','valid_loss','valid_accs'])


df['layers'] = 1

#adjust activation function
for i in range(1,7):
    df.at[i,'activations'] = [relu, relu, relu, relu]
for i in range(7,13):
    df.at[i,'activations'] = [tanh, tanh, tanh, tanh]
IMAGE_SHAPE = [73,60,3]
df['conv_out_channels'] = 16
df['kernel_size'] = 5
df['conv_stride'] = 1
df['maxpool'] = 1
df['dropout'] = 0.0
df['batchnorm'] = False
df['optimizer']='Adam'
df['learning_rate']=0.001
df['weight_decay']=0.0
df['batch_size']=128
df['num_epochs']=5

#adjust depth
df.at[5:6,'layers'] = 2
df.at[11:12,'layers'] = 2


#adjust channels
df.at[2,'conv_out_channels'] = 32 
df.at[3,'conv_out_channels'] = 64
df.at[4,'conv_out_channels'] = 128
df.at[5,'conv_out_channels'] = 32
df.at[6,'conv_out_channels'] = 64
df.at[8,'conv_out_channels'] = 32 
df.at[9,'conv_out_channels'] = 64
df.at[10,'conv_out_channels'] = 128
df.at[11,'conv_out_channels'] = 32
df.at[12,'conv_out_channels'] = 64 

In [ ]:
for i in range(5,13):
    print('Model: ', i)
    layers = df.loc[i,'layers']
    activations = df.loc[i,'activations']
    conv_out_channels = df.loc[i,'conv_out_channels']
    kernel_size = df.loc[i,'kernel_size']
    conv_stride = df.loc[i,'conv_stride']
    maxpool = int(df.loc[i,'maxpool'])
    dropout = df.loc[i,'dropout']
    batchnorm = df.loc[i,'batchnorm']
    optimizer = df.loc[i,'optimizer']
    learning_rate = df.loc[i,'learning_rate']
    weight_decay = df.loc[i,'weight_decay']
    batch_size = int(df.loc[i,'batch_size'])
    num_epochs = df.loc[i,'num_epochs']
    
    net = network_tuning.tune_architecture(layers, activations, IMAGE_SHAPE, conv_out_channels, kernel_size,conv_stride, maxpool, dropout, batchnorm)
    net_trained, df.at[i,'train_loss'], df.at[i,'train_accs'] = network_tuning.tune_train(net, data_train, optimizer, learning_rate, weight_decay, batch_size, num_epochs)
    df.at[i,'valid_loss'], df.at[i,'valid_accs'] = network_tuning.tune_valid(net, data_valid, batch_size)
    df.at[i,'net'] = 'model'+str(i)
    df.to_pickle('df.pkl')
    torch.save(net_trained.state_dict(), 'model'+str(i))

Model:  5
No GPU available.
Epoch:  0


In [ ]:
plt.plot(pd.Series(df.loc[4,'train_accs']).rolling(window=600).mean())

In [ ]:
pd.Series(df.loc[1,'train_accs']).rolling(window=600).mean()

In [ ]:
torch.cuda.current_device()

In [ ]:
torch.cuda.device(0)
torch.cuda.device_count()
torch.cuda.get_device_name(0)

In [ ]:
torch.cuda.device_count()

In [ ]:
torch.backends.cudnn.enabled

In [ ]:
import nvidia

In [ ]:
!apt-get install cuda-drivers

In [ ]:
!nvidia-smi

In [ ]:
!nvcc --version

In [ ]:
! curl -O http://us.download.nvidia.com/tesla/384.125/nvidia-diag-driver-local-repo-ubuntu1604-384.125_1.0-1_amd64.deb

In [ ]:
!sudo dpkg -i apt_1.4_amd64.deb

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [ ]:
torch.cuda.is_available()